In [8]:
import pandas as pd
import numpy as np

In [ ]:
class Event() {
    
}

In [21]:
def generate_perlin_noise(shape, scale=10, threshold=0):
    noise = PerlinNoise(octaves=4)  
    noise_array = np.zeros(shape)
    
    x = np.linspace(0, 1, shape[0]) 
    y = np.linspace(0, 1, shape[1]) 
    xv, yv = np.meshgrid(x, y, indexing='ij')  
    
    for i in range(shape[0]):
        for j in range(shape[1]):
            noise_value = noise([xv[i, j] * scale, yv[i, j] * scale])
            noise_array[i, j] = 1 if noise_value > threshold else 0
    
    return noise_array

exampleSchedule = np.asarray([generate_perlin_noise((7, 135), scale=5, threshold=0) for _ in range(15)])   ################## Generates 15 example schedules to use for testing

groups = [np.asarray([5,7,9,1,11,13]),np.asarray([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])] ###################### define groups


print(exampleSchedule[0])

[[0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.
  0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1.
  1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0.
  0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1.
  1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1.
  1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1.
  1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.
  1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
  0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1.
  1. 1. 1. 1. 1.

In [22]:
def scheduleMix(Schedules: list[np.ndarray], group: list[int]) -> list[np.ndarray]: 
    return np.sum(Schedules[group], axis = 0)


In [24]:
scheduleMix(exampleSchedule, groups[0])

array([[0., 0., 1., 1., 1., 2., 3., 3., 3., 1., 1., 1., 1., 2., 4., 4.,
        4., 3., 3., 2., 2., 4., 4., 4., 2., 3., 4., 2., 2., 3., 3., 3.,
        2., 2., 4., 4., 2., 3., 3., 3., 2., 4., 3., 2., 2., 3., 3., 3.,
        3., 3., 3., 4., 3., 4., 2., 2., 2., 2., 2., 2., 2., 4., 5., 5.,
        5., 5., 5., 0., 1., 1., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3.,
        2., 3., 3., 3., 2., 1., 2., 2., 4., 4., 4., 4., 3., 2., 4., 4.,
        2., 2., 3., 4., 3., 3., 3., 3., 3., 2., 1., 1., 5., 5., 4., 4.,
        4., 4., 2., 2., 2., 3., 5., 4., 4., 2., 2., 2., 4., 4., 4., 4.,
        2., 2., 2., 1., 1., 0., 0.],
       [1., 2., 3., 4., 3., 3., 3., 4., 4., 3., 3., 3., 2., 3., 2., 2.,
        1., 1., 1., 0., 0., 3., 3., 2., 3., 2., 2., 1., 2., 2., 3., 3.,
        3., 3., 3., 3., 3., 3., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3.,
        2., 2., 2., 2., 3., 4., 5., 6., 5., 3., 2., 2., 2., 1., 4., 4.,
        4., 4., 4., 4., 4., 5., 4., 3., 3., 3., 4., 4., 4., 4., 3., 3.,
        3., 4., 3., 3., 3.,